In [2]:
import pandas as pd
from ensemble_boxes import *
import numpy as np
from pycocotools.coco import COCO
import cv2
import matplotlib.pyplot as plt
import copy

In [3]:
submission_files = ['/opt/ml/level2_objectdetection_cv-level2-cv-15/mmdetection/submission_hj_1.csv',
                   '/opt/ml/level2_objectdetection_cv-level2-cv-15/mmdetection/submission_sy_1.csv',
    ]
submission_df = [pd.read_csv(file) for file in submission_files]


In [4]:
image_ids = submission_df[0]['image_id'].tolist()

In [5]:
# ensemble 할 file의 image 정보를 불러오기 위한 json
annotation = '/opt/ml/dataset/test.json'
coco = COCO(annotation)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [6]:
prediction_strings = []
file_names = []
# ensemble 시 설정할 iou threshold 
# 수정하면서 실습 하시면 됩니다!
iou_thr = 0.55

# 각 image id 별로 submission file에서 box좌표 추출
for i, image_id in enumerate(image_ids):
    prediction_string = ''
    boxes_list = []
    scores_list = []
    labels_list = []
    image_info = coco.loadImgs(i)[0]
# 각 submission file 별로 prediction box좌표 불러오기
    for df in submission_df:
        predict_string = df[df['image_id'] == image_id]['PredictionString'].tolist()[0]
        predict_list = str(predict_string).split()
        
        if len(predict_list)==0 or len(predict_list)==1:
            continue
            
        predict_list = np.reshape(predict_list, (-1, 6))
        box_list = []
        
        for box in predict_list[:, 2:6].tolist():
            # bounding box 좌표값들이 0~1 사이의 값을 가져야 합니다.
            # Order: x1, y1, x2, y2
            box[0] = float(box[0]) / image_info['width']
            box[1] = float(box[1]) / image_info['height']
            box[2] = float(box[2]) / image_info['width']
            box[3] = float(box[3]) / image_info['height']
            box_list.append(box)
            
        boxes_list.append(box_list)
        scores_list.append(list(map(float, predict_list[:, 1].tolist())))
        labels_list.append(list(map(int, predict_list[:, 0].tolist())))
    
# 예측 box가 있다면 이를 ensemble 수행
# wbf 외에도 다른 ensemble을 사용할 수 있습니다
    if len(boxes_list):
        # boxes, scores, labels = nms(boxes_list, scores_list, labels_list, iou_thr=iou_thr)
        # boxes, scores, labels = soft_nms(boxes_list, scores_list, labels_list, iou_thr=iou_thr)
        # boxes, scores, labels = non_maximum_weighted(boxes_list, scores_list, labels_list, iou_thr=iou_thr)
        boxes, scores, labels = weighted_boxes_fusion(boxes_list, scores_list, labels_list, iou_thr=iou_thr)
        
        for box, score, label in zip(boxes, scores, labels):
            prediction_string += str(int(label)) + ' ' + str(score) + ' ' + str(box[0] * image_info['width']) + ' ' + str(box[1] * image_info['height']) + ' ' + str(box[2] * image_info['width']) + ' ' + str(box[3] * image_info['height']) + ' '
    
    prediction_strings.append(prediction_string)
    file_names.append(image_id)

In [8]:
submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv('/opt/ml/level2_objectdetection_cv-level2-cv-15/mmdetection/ensemble_2.csv', index=None)

submission.head()

,PredictionString,image_id
0,7 0.99960857629776 216.52163696289062 49.83084...,test/0000.jpg
1,4 0.44195157289505005 343.8524169921875 245.74...,test/0001.jpg
2,1 0.49623429775238037 86.14695739746094 277.73...,test/0002.jpg
3,9 0.35754016041755676 153.71739196777344 261.7...,test/0003.jpg
4,1 0.31303316354751587 204.0697021484375 257.96...,test/0004.jpg
